# Load Data

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [27]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_torino'

md = metadata_loader(_extract=True, _drop_inutile=True)
# md = md[md["job_status"] == "JobStatus.DONE"]
# md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]


md = md[md["shots"]==10000]
md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,rep_delay,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
2528,2023-12-02 11:16:20.667000+01:00,torino_repcode,ibm_torino,cnng7x70k0p00088v8vg,[RepCode],10000.0,NaN,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,1,"_is_hex=True, sherbrooke",diff rounds v2
2527,2023-12-02 11:16:18.322000+01:00,torino_repcode,ibm_torino,cnng7wq24wx0008fagdg,[RepCode],10000.0,NaN,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,"_is_hex=True, sherbrooke",diff rounds v2


In [6]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_0"
memory = memories[memory_str][:]
job_id = jobs[memory_str]
print(memory.shape)

(10000, 100)


# Decode using get counts real part

In [32]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid

d = max_distance
synd_rounds = max_distance
synd_rounds = 10

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=synd_rounds,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)

# layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)

##### TORINO ####
distance = max_distance
import pickle
with open('longest_path_torino.pkl', 'rb') as f:
    path = pickle.load(f)
bounded_path = path[:2 * distance - 1]
layout = bounded_path[1::2] + bounded_path[::2] 


qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes
# print(qubit_mapping)
# print(len(qubit_mapping))

In [33]:
from tqdm import tqdm
from soft_info import counts_to_det_syndr, draw_matching_graph, get_counts, reweight_edges_to_one

VERBOSE = False

actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []

matching = pymatching.Matching.from_detector_error_model(model)
reweight_edges_to_one(matching)

for shot in tqdm(range(len(memory))[:]):
    
    i += 1
    IQ_data = memory[shot]
    # IQ_data_reshaped = IQ_data.reshape(1, -1)
    IQ_data_reshaped = IQ_data

    counts = get_counts([IQ_data_reshaped])
    count_key = next(iter(counts.keys()))

    # if count_key == '000 00 00 00':
        #print("Skipping all zeros")
        # continue    

    # cpp_soft_info.soft_reweight_pymatching(matching._matching_graph, IQ_data_reshaped, synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict, 6e-3, 1e-70, 0.1) 
    
    # print(count_key)
    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False, distance=10)

    predicted_observables = matching.decode(array_processed_string)

    actual_observables = [(int(count_key[0])+0)%2]
    if predicted_observables == actual_observables: #== [0]:
        continue
    num_errors += 1
    # print("num_errors:", num_errors)
    
    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count str to syndromes:", array_processed_string)


    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        print("Actual flip:", actual_observables)
    

    if VERBOSE:
        draw_matching_graph(matching, d, synd_rounds, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(6, 6))

print("Num errors:", num_errors)

100%|██████████| 10000/10000 [00:01<00:00, 6291.98it/s]

Num errors: 14
